<a href="https://colab.research.google.com/github/MoritaHiroki87/ZZ_MachineLearningNote/blob/master/Chainer%E5%85%A5%E9%96%80%EF%BC%88%E5%9B%9E%E5%B8%B0%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7 Chainer入門（回帰）

# 7.1 前準備

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import chainer
import chainer.links as L
import chainer.functions as F

# 7.2 データセットの準備

In [127]:
df = pd.read_csv('sample_data/california_housing_train.csv')
df.shape

(17000, 9)

## 7.2.1 入出力の分割

In [0]:
x = df.iloc[:, :-1]
t = df.iloc[:, -1]

## 7.2.2 numpyデータにして、TupleDatasetに変換

注意：回帰の問題ではtargetのデータの形が(506, )とかではだめらしい。(506, 1)に直す必要あり。   
なんでだ？？？？

In [0]:
x = np.array(x, dtype='float32')
t = np.array(t, dtype='float32')
t = t.reshape(len(t), 1)

In [130]:
print(x.shape)
print(type(x))
print(type(x[0, 0]))

print(t.shape)
print(type(t))
print(type(t[0, 0]))
print(t[0, 0])

(17000, 8)
<class 'numpy.ndarray'>
<class 'numpy.float32'>
(17000, 1)
<class 'numpy.ndarray'>
<class 'numpy.float32'>
66900.0


In [0]:
dataset = chainer.datasets.TupleDataset(x, t)
# dataset = list(zip(x, t))

### 7.2.3 データを訓練、検証、テストに分割

In [0]:
n_train = int(len(dataset) * 0.7)
n_valid = int(len(dataset) * 0.2) # ここの大きさの設定を地味にミスる

train, valid_test = chainer.datasets.split_dataset_random(dataset, n_train, seed=0)
valid, test = chainer.datasets.split_dataset_random(valid_test, n_valid, seed=0)

# 7.3 ネットワークの定義

回帰は出力層のノード数が１。   
インスタンス化する前にシードの固定をする。これわからんな！

## 7.3.1 モデルだけ定義

In [0]:
class NN(chainer.Chain):
    
    def __init__(self, n_mid_units=5, n_out=1):
        super().__init__()
        with self.init_scope():
            self.fc1 = L.Linear(None, n_mid_units)
            self.fc2 = L.Linear(None, n_out)
            
    def __call__(self, x):
        h = self.fc1(x)
        h = F.relu(h)
        h = self.fc2(h)
        return h

In [0]:
np.random.seed(0)

In [0]:
nn = NN()

## 7.3.2 損失関数の定義

疑問：   
損失関数はLなのか？　Fでは？　いやでも線形ではあるがパラメータないよな？   
あとこれどこでtの値をセットしてんだ？　損失関数回すには必要だと思うが。。

注意：回帰では精度みたいなのが取れないんで``model.compute_accuracy``を``False``にする。

In [0]:
model = L.Classifier(nn, lossfun=F.mean_squared_error)
model.conpute_accuracy = False

# 7.4 オプティマイザの選定

In [137]:
optimizer = chainer.optimizers.SGD()
optimizer.setup(model)

# 7.5 イテレータの作成
疑問：   
これ検証データでリピートとシャッフルを打ち消してるのなんでなんだろ。シャッフルは必要ないって思う、リピートはエポック数１に固定したいってこと？

In [0]:
batchsize = 10
train_iter = chainer.iterators.SerialIterator(train, batchsize)
valid_iter = chainer.iterators.SerialIterator(valid, batchsize, repeat=False, shuffle=False)

# 7.6 アップデータの作成

In [0]:
from chainer import training

In [0]:
updater = training.StandardUpdater(train_iter, optimizer, device=-1)

# 7.7 トレーナの作成

In [0]:
epoch = 50
trainer = training.Trainer(updater, (epoch, 'epoch'), out='result/housing')

# 7.8 エクステンションの設定

In [0]:
from chainer.training import extensions

trainer.extend(extensions.Evaluator(valid_iter, model, device=-1))

trainer.extend(extensions.LogReport(trigger=(1, 'epoch')))

trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'validation/main/loss', 'elapsed_time']), trigger=(1, 'epoch'))

# 7.9 訓練実行！

In [143]:
trainer.run()

Exception in main training loop: 
Invalid operation is performed in: Accuracy (Forward)

Expect: t.dtype.kind == i
Actual: f != i
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/chainer/training/trainer.py", line 315, in run
    update()
  File "/usr/local/lib/python3.6/dist-packages/chainer/training/updaters/standard_updater.py", line 165, in update
    self.update_core()
  File "/usr/local/lib/python3.6/dist-packages/chainer/training/updaters/standard_updater.py", line 177, in update_core
    optimizer.update(loss_func, *in_arrays)
  File "/usr/local/lib/python3.6/dist-packages/chainer/optimizer.py", line 680, in update
    loss = lossfun(*args, **kwds)
  File "/usr/local/lib/python3.6/dist-packages/chainer/link.py", line 242, in __call__
    out = forward(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/chainer/links/model/classifier.py", line 147, in forward
    self.accuracy = self.accfun(self.y, t)
  File "/usr/local/lib/python3.6/

InvalidType: ignored

# 7.10 練習問題

平均二乗誤差が検証データで大きい。どうする。
なんで大きくなるんだろうね。またあれか？　入力データのスケールを修正するか？